# DQN notebook

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import neat
import matplotlib.pyplot as plt
from matplotlib import cm

import pickle
import multimodal_mazes
from tqdm import tqdm

import itertools
import torch
import torch.nn as nn
import torch.optim as optim

## Ideas 
* Optuna: https://colab.research.google.com/github/araffin/tools-for-robotic-rl-icra2022/blob/main/notebooks/optuna_lab.ipynb#scrollTo=E0yEokTDxhrC 

## Working

In [ ]:
wm_flags = np.array(list(itertools.product([0,1], repeat=7)))
agnt = multimodal_mazes.AgentDQN(location=[5,5], channels=[1,1], sensor_noise_scale=0.05, n_hidden_units=8, wm_flags=wm_flags[0])
agnt

In [ ]:
maze = multimodal_mazes.TrackMaze(size=11, n_channels=2)

maze.generate(number=60000, noise_scale=0.0, gaps=0)
# agnt.generate_policy(maze, n_steps=6)

In [ ]:
plt.imshow(maze.mazes[100][:,:,1])

In [ ]:
n = 0
time, path = multimodal_mazes.maze_trial(mz=maze.mazes[n], mz_start_loc=maze.start_locations[n], mz_goal_loc=maze.goal_locations[n], channels=[1,1], sensor_noise_scale=0.05, drop_connect_p=0.0, n_steps=6, agnt=agnt)
print(time, path)

multimodal_mazes.plot_path(path, mz=maze.mazes[n], mz_goal_loc=maze.goal_locations[n], n_steps=6)

In [ ]:
maze = multimodal_mazes.TrackMaze(size=11, n_channels=2)
maze.generate(number=1000, noise_scale=0.0, gaps=0)

fitness = multimodal_mazes.eval_fitness(genome=None, config=None, channels=[1,1], sensor_noise_scale=0.05, drop_connect_p=0.0, maze=maze, n_steps=6, agnt=agnt)
print(fitness)

In [ ]:
plt.plot(agnt.gradient_norms)

In [ ]:
plt.imshow(agnt.output_to_output.weight.detach())
plt.colorbar()

## Fitness vs noise

In [ ]:
# Fitness vs noise
noises = np.linspace(start=0.0, stop=0.5, num=21)

# Small set
wm_flags = np.array([[0,0,0,0,0,0,0], [0,0,0,0,0,0,0], [1,1,1,1,0,0,0],[1,1,1,1,1,1,1]])
colors = ["xkcd:gray", [0.0, 0.0, 0.0, 0.5], [0.039, 0.73, 0.71, 1], list(np.array([24, 156, 196, 255]) / 255)]

# Full set 
# wm_flags = np.array(list(itertools.product([0,1], repeat=7)))
# wm_flags = np.vstack((wm_flags[0], wm_flags))
# colors = cm.get_cmap("plasma", len(wm_flags)).colors.tolist()

results = np.zeros((len(noises), len(wm_flags)))

# Generate mazes
maze = multimodal_mazes.TrackMaze(size=11, n_channels=2)
maze.generate(number=100000, noise_scale=0.0, gaps=2)

maze_test = multimodal_mazes.TrackMaze(size=11, n_channels=2)
maze_test.generate(number=1000, noise_scale=0.0, gaps=2)

# Run
for b, wm_flag in enumerate(tqdm(wm_flags)): 

    # Control architecture 
    if b != 1: 
        n_hidden_units = 8
    else:
        n_hidden_units = 34 
    
    # Train
    agnt = multimodal_mazes.AgentDQN(location=[5,5], channels=[1,1], sensor_noise_scale=0.05, n_hidden_units=n_hidden_units, wm_flags=wm_flag)
    agnt.generate_policy(maze, n_steps=6) 

    # Test 
    for a, noise in enumerate(noises):
        results[a,b] = multimodal_mazes.eval_fitness(genome=None, config=None, channels=[1,1], sensor_noise_scale=noise, drop_connect_p=0.0, maze=maze_test, n_steps=6, agnt=agnt)

# Plotting
plt.plot([0.05, 0.05], [0,1], ':', color='k', alpha=0.5, label='Training noise')
for b, wm_flag in enumerate(wm_flags): 
    plt.plot(noises, results[:,b], color=colors[b], label=wm_flag)

plt.ylim([0, 1.05])
plt.ylabel('Fitness')
plt.xlabel('Sensor noise')
plt.legend()

In [ ]:
# Fitness vs noise AUC 
auc = np.trapz(y=results.T, x=noises, axis=1)
idxs = np.argsort(auc)

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(5*3,5), sharex=False, sharey=True)
for b, idx in enumerate(idxs): 
    ml, sl, _ = plt.stem(b, auc[idx])
    ml.set_color('k')
    sl.set_color('k')
# plt.xticks(range(len(wm_flags)), policies, rotation='vertical')
plt.ylabel('AUC');

## Loading

In [ ]:
import os
paths = ['../Results/test' + str(n) + '/' for n in range(58,68)]
print(paths)

noises = np.linspace(start=0.0, stop=0.5, num=21) # noises,
results = np.zeros((len(noises), 129, len(paths))) * np.nan # noises x architectures x repeats
wm_flags = np.zeros((129, 7, len(paths))) * np.nan # architectures x flags x repeats
n_parameters = np.zeros((129, len(paths))) * np.nan # architectures x repeats
auc = np.zeros((129, len(paths))) * np.nan # architectures x repeats 

for a, path in enumerate(tqdm(paths)):

    # Load data 
    for f in os.listdir(path):
        if f.endswith(".pickle"):
            with open(path + f, 'rb') as file:
                agnt = pickle.load(file)
                idx = int(os.path.splitext(f)[0])

                results[:, idx, a] = agnt.results
                wm_flags[idx, :, a] = agnt.wm_flags
                n_parameters[idx, a] = agnt.n_parameters
                auc[idx, a] = np.trapz(y=agnt.results, x=noises)

In [ ]:
# Graph features 
import networkx as nx
from scipy.linalg import expm

Gs, communcability, n_cycles, cycle_c = [], [], [], []
for a, wm_flag in enumerate(np.nanmax(wm_flags, axis=2)):
    G = nx.DiGraph([(0,1), (1,2)]) # F0 and F1 
    if wm_flag[0]: G.add_edge(0,0) # L0
    if wm_flag[1]: G.add_edge(1,1) # L1
    if wm_flag[2]: G.add_edge(2,2) # L2
    if wm_flag[3]: G.add_edge(0,2) # S0
    if wm_flag[4]: G.add_edge(2,0) # S1
    if wm_flag[5]: G.add_edge(1,0) # B0
    if wm_flag[6]: G.add_edge(2,1) # B1

    Gs.append(G)
    communcability.append(np.mean(expm(nx.adjacency_matrix(G).toarray())))
    n_cycles.append(len(list(nx.simple_cycles(G))))
    try:
        cycle_c.append(max([len(c) for c in nx.simple_cycles(G)]))
    except: 
        cycle_c.append(0)

In [ ]:
# Sorting
idxs = np.argsort(np.nanmax(auc,axis=1) - np.nanstd(auc,axis=1)) # Sort by max minus std  

# Networks of interest
interest = [1,128, np.argmax(np.nanmean(results[2,:,:], axis=1)), np.argmax(np.nanmean(auc, axis=1))]
print(interest)
i_cols = ['xkcd:dusty blue', 'xkcd:purple', 'xkcd:dark seafoam', 'xkcd:yellowish']
i_labels = ['Feedforward (L)', 'Fully recurrent', 'Most accurate', 'Most robust']

In [ ]:
# AUC vs paramters (scatters and curve fits)
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(5*3,5), sharex=False, sharey=True)

for a in range(3):
    plt.sca(ax[a])
    
    if a == 0: 
        x = results[2,:,:].reshape(-1) # networks,
        plt.xlabel('Test accuracy')
        plt.ylabel('AUC')

    if a == 1: 
        x = n_parameters.reshape(-1) # networks,
        plt.xlabel('N parameters')

    if a == 2: 
        x = np.repeat(communcability, repeats=len(paths)) # networks
        plt.xlabel('Mean communcability')

    y = auc.reshape(-1) # networks,
    z = np.repeat(np.arange(start=0, stop=129), repeats=len(paths)) # networks

    x = x[np.isnan(y) == False]
    z = z[np.isnan(y) == False]
    y = y[np.isnan(y) == False]

    idx = np.argsort(x)

    curve = np.poly1d(np.polyfit(x[idx],y[idx],deg=2))
    plt.plot(x[idx], curve(x[idx]), color='xkcd:grey', alpha=0.5)

    plt.scatter(x, y, s=10, color='k', marker='.', alpha=0.2, label='All networks')

    for b, i in enumerate(interest):
        plt.scatter(x[z == i], y[z == i], color=i_cols[b], marker='.', alpha=0.75, label=i_labels[b])
    
    if a == 0:
        plt.legend()

In [ ]:
# Fitness vs noise AUC 
from matplotlib.patches import Rectangle

wm_f_labels = ['L0', 'L1', 'L2', 'S0', 'S1', 'B0', 'B1']

fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(5*3,5), sharex=True, sharey=False)

plt.sca(ax[0])
plt.plot([range(129), range(129)], [np.nanmin(auc,axis=1)[idxs], np.nanmax(auc,axis=1)[idxs]], color='k', alpha=0.25);
plt.ylabel('AUC');
plt.xticks([])

plt.sca(ax[1])
for i in range(7):
    plt.scatter(range(129), np.ones(129) * i, c=[(0, 0, 0, alpha) for alpha in np.nanmax(wm_flags, axis=2)[idxs,i]], s=5)
plt.yticks(range(7), wm_f_labels)
plt.xlabel('Architectures')

for a, i in enumerate(interest):
    plt.sca(ax[0])
    plt.plot([np.where(idxs == i)[0], np.where(idxs == i)[0]], [np.nanmin(auc[i]), np.nanmax(auc[i])], color=i_cols[a])  

    ax[1].add_patch(Rectangle((np.where(idxs == i)[0][0] -0.25, -0.95), 0.5, 7.05, color=i_cols[a], alpha=0.5))

In [ ]:
# Fitness vs noise 
plt.plot([0.05, 0.05], [0,1], ':', color='k', alpha=0.5, label='Training noise')

plt.plot([], [], 'k', alpha=0.1, label='All architectures')
plt.plot(noises, results[:,range(129), np.nanargmax(auc,axis=1)], c='k', alpha=0.1);

for a, i in enumerate(interest):
    plt.plot(noises, results[:, i, np.nanargmax(auc[i])], color=i_cols[a], label=i_labels[a])

plt.ylim([0, 1.05])
plt.ylabel('Fitness')
plt.xlabel('Sensor noise')
plt.legend()

In [ ]:
# Fitness vs noise 
plt.plot([0.05, 0.05], [0,1], ':', color='k', alpha=0.5, label='Training noise')

for a, i in enumerate(interest):
    plt.plot(noises, results[:, i], color=i_cols[a], label=i_labels[a], alpha=0.5)

plt.ylim([0, 1.05])
plt.ylabel('Fitness')
plt.xlabel('Sensor noise')

## Plotting architectures

In [ ]:
# Graph features 
import numpy as np
import networkx as nx

Gs = []
for a, wm_flag in enumerate(np.nanmax(wm_flags, axis=2)):
    G = nx.DiGraph([(0,1), (1,2)]) # F0 and F1 
    if wm_flag[0]: G.add_edge(0,0) # L0
    if wm_flag[1]: G.add_edge(1,1) # L1
    if wm_flag[2]: G.add_edge(2,2) # L2
    if wm_flag[3]: G.add_edge(0,2) # S0
    if wm_flag[4]: G.add_edge(2,0) # S1
    if wm_flag[5]: G.add_edge(1,0) # B0
    if wm_flag[6]: G.add_edge(2,1) # B1

    Gs.append(G)

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(nrows=8, ncols=16, figsize=(30,15), sharex=True, sharey=True)

for a, _ in enumerate(ax.ravel()):
    plt.sca(ax.ravel()[a])

    wm_flag = np.nanmax(wm_flags,axis=2)[a+1]
    multimodal_mazes.plot_dqn_architecture(wm_flag)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1)
multimodal_mazes.plot_dqn_architecture(np.nanmax(wm_flags,axis=2)[36])
new_color = 'xkcd:yellowish'

for line in ax.get_lines(): 
    line.set_color(new_color)

for patch in ax.patches: 
    patch.set_facecolor(new_color)
    patch.set_edgecolor(new_color)

for path_collection in ax.collections:
    path_collection.set_facecolor(new_color)
    path_collection.set_edgecolor(new_color)


## Exploring function

### Loading networks 

In [ ]:
import os
import copy 
paths = ['../Results/test' + str(n) + '/' for n in range(58,68)]
print(paths)

interest = [1, 128, 74, 36]
i_cols = ['xkcd:dusty blue', 'xkcd:purple', 'xkcd:dark seafoam', 'xkcd:yellowish']
i_labels = ['Feedforward (L)', 'Fully recurrent', 'Most accurate', 'Most robust']

agents, idxs = [], []
for a, path in enumerate(tqdm(paths)):

    # Load data 
    for f in interest:
        try:
            with open(path + str(f) + ".pickle", 'rb') as file:
                    agnt = pickle.load(file)
                    agents.append(agnt)
                    idxs.append(f)
        except: 
            pass

print(np.unique(idxs, return_counts=True))

### Sensitivity

In [ ]:
s_min = 0.0 
s_max = 0.5 
s_num = 49 # must be uneven
n_steps = 6

sensitivity_inputs = [np.array([np.linspace(start=s_min, stop=s_max, num=s_num)[::-1], np.linspace(start=s_min, stop=s_max, num=s_num)[::-1], 
                    np.linspace(start=s_min, stop=s_max, num=s_num), np.linspace(start=s_min, stop=s_max, num=s_num)]) for _ in range(2)]

sensitivity_inputs[0][0, (s_num // 2) + 1:] = 0.0
sensitivity_inputs[0][2, :(s_num // 2)] = 0.0

sensitivity_inputs[1][1, (s_num // 2) + 1:] = 0.0
sensitivity_inputs[1][3, :(s_num // 2)] = 0.0

In [ ]:
for a, i in enumerate(interest): # for each architecture
    L_outputs = []
    for i_num, v in enumerate(np.where(np.array(idxs) == i)[0]): # for each network
        agnt = copy.deepcopy(agents[v])
        for _, s_m in enumerate(sensitivity_inputs):
            l_outputs = []
            for _, s_i in enumerate(s_m.T):

                # Reset agent
                agnt.prev_input *= 0.0
                agnt.hidden *= 0.0
                agnt.prev_output *= 0.0
                agnt.outputs *= 0.0
                agnt.channel_inputs *= 0.0    

                for _ in range(n_steps):
                    agnt.channel_inputs[0] = np.copy(s_i[:2])
                    agnt.channel_inputs[1] = np.copy(s_i[2:])
                    agnt.policy()
                l_outputs.append(np.copy(agnt.outputs[0]))

            L_outputs.append(l_outputs)

    x = range(s_num)
    y = np.nanmean(np.array(L_outputs),axis=0)
    sem = np.nanstd(np.array(L_outputs),axis=0) / np.sqrt(i_num + 1)
    plt.plot(x, y, color=i_cols[a], label=i_labels[a])
    plt.fill_between(x, y-sem, y+sem, color=i_cols[a], alpha=0.25, edgecolor=None)

plt.ylim([-0.05,1.05])
plt.legend()
plt.xticks(ticks=[0, int(max(x)/2), max(x)], labels=("1.0", "0.5", "0.0"))
plt.xlabel('Left input')
plt.ylabel("p(Left)")

### Sensitivity (Jacobian)

In [ ]:
from torch.autograd.functional import jacobian

In [ ]:
def forward(x):
    # Reset agent
    agnt.prev_input = torch.zeros(agnt.n_input_units)
    agnt.hidden = torch.zeros(agnt.n_hidden_units)
    agnt.prev_output = torch.zeros(agnt.n_output_units)
    agnt.outputs = torch.zeros(agnt.n_output_units)
    
    for t in range(2):
        agnt.channel_inputs = x
        agnt.outputs, agnt.prev_input, agnt.hidden, agnt.prev_output = agnt.forward(
                agnt.prev_input, agnt.hidden, agnt.prev_output, tensor_input=True)    
    return agnt.outputs

In [ ]:
for a, i in enumerate(interest): # for each architecture
    L_outputs = []
    for i_num, v in enumerate(np.where(np.array(idxs) == i)[0]): # for each network
        agnt = copy.deepcopy(agents[v])
        for _, s_m in enumerate(sensitivity_inputs):
            l_outputs = []
            for _, s_i in enumerate(s_m.T):

                x = torch.from_numpy(np.concatenate((s_i, np.zeros(4)))).to(torch.float32)
                jm = jacobian(forward, x)
                f_norm = torch.norm(jm, p='fro')
                l_outputs.append(f_norm)

            L_outputs.append(l_outputs)

    x = range(s_num)
    y = np.nanmean(np.array(L_outputs),axis=0)
    sem = np.nanstd(np.array(L_outputs),axis=0) / np.sqrt(i_num + 1)
    plt.plot(x, y, color=i_cols[a], label=i_labels[a])
    plt.fill_between(x, y-sem, y+sem, color=i_cols[a], alpha=0.25, edgecolor=None)

plt.ylim([-0.05,10.05])
plt.legend()
plt.xticks(ticks=[0, int(max(x)/2), max(x)], labels=("1.0", "0.5", "0.0"))
plt.xlabel('Left input')
plt.ylabel("Frobenius norm")

### Memory

In [ ]:
L_input = [0.1,0.1]
R_inputs = [[0.1, 0.0], [0.0, 0.1]]

for a, i in enumerate(interest): # for each architecture 
    L_outputs = []
    for i_num, v in enumerate(np.where(np.array(idxs) == i)[0]): # for each network
        agnt = copy.deepcopy(agents[v])

        for b in range(2):
            l_outputs = [] 
            
            # Reset agent
            agnt.prev_input *= 0.0
            agnt.hidden *= 0.0
            agnt.prev_output *= 0.0
            agnt.outputs *= 0.0
            agnt.channel_inputs *= 0.0    

            for t in range(6):

                if t == 0:
                    agnt.channel_inputs[0] = np.copy(L_input)
                    agnt.channel_inputs[1] = np.copy(R_inputs[b])
                else: 
                    agnt.channel_inputs *= 0.0

                agnt.policy()
                l_outputs.append(np.copy(agnt.outputs[0]))
            L_outputs.append(l_outputs)

    x = range(np.array(L_outputs).shape[1])
    y = np.nanmean(np.array(L_outputs),axis=0)
    sem = np.nanstd(np.array(L_outputs),axis=0) / np.sqrt(i_num + 1)
    plt.plot(x, y, color=i_cols[a], label=i_labels[a])
    plt.fill_between(x, y-sem, y+sem, color=i_cols[a], alpha=0.25, edgecolor=None)

plt.ylim([-0.05,1.05])
plt.legend()
plt.xlabel('Time since input')
plt.ylabel("p(Left)")

### Noise suppression

In [ ]:
for a, i in enumerate(interest): # for each architecture 
    L_outputs = []
    for i_num, v in enumerate(np.where(np.array(idxs) == i)[0]): # for each network
        agnt = copy.deepcopy(agents[v])

        for _ in range(100):
            l_outputs = [] 
            
            # Reset agent
            agnt.prev_input *= 0.0
            agnt.hidden *= 0.0
            agnt.prev_output *= 0.0
            agnt.outputs *= 0.0
            agnt.channel_inputs *= 0.0    

            for t in range(6):
                agnt.channel_inputs = np.random.normal(
                    loc=0.0, scale=0.05, size=agnt.channel_inputs.shape
                )
                agnt.channel_inputs = np.clip(agnt.channel_inputs, a_min=0.0, a_max=1.0)
                agnt.policy()
                l_outputs.append(np.std(np.array(agnt.outputs)))
            L_outputs.append(l_outputs)

    x = range(np.array(L_outputs).shape[1])
    y = np.nanmean(np.array(L_outputs),axis=0)
    sem = np.nanstd(np.array(L_outputs),axis=0) / np.sqrt(i_num + 1)
    plt.plot(x, y, color=i_cols[a], label=i_labels[a])
    plt.fill_between(x, y-sem, y+sem, color=i_cols[a], alpha=0.25, edgecolor=None)

plt.ylim([-0.05,0.45])
plt.legend()
plt.xlabel('Time with noise input')
plt.ylabel("Std(p(outputs))")